# Seq2seq для машинного перевода

В этом блокноте рассматриваются некоторые подходы к задаче машинного перевода с помощью

* Рекуррентных сетей
* Рекуррентных сетей с механизмом внимания
* Трасформеров

Первый и последний подходы уже реализованы. Вам предлагается реализовать второй подход, а именно интегрировать механизм внимания в рекуррентную НС. Для лучшего понимания рекомендуем ознакомиться [со следющей статьей](https://arxiv.org/pdf/1409.0473). Какой конкретно тип механизма внимания реализовывать остается на выбор студенту.

Для оценок трех рассмотренных подходов реализовать метрику BLEU. В качестве тестовой выборки можно использовать валидационный набор. Или произвести требуемое разделение самостоятельно.

В качестве резюме (на 3 балла):

1. Разобраться в задаче и в коде
2. Добавить внимание к рекуррентной сети
3. Реализовать BLEU
4. Сравнить полученные 3 модели между собой.
5. **Опционально (+1 балл)**: продемонстрировать alignment между словами на исходном и целевом языках (аналогично Figure 3 в предложенной статье).
6. **Опционально (+1 балл)**: сравнить 3 полученных модели (по метрике) между собой на парах различной длины. Например, вычислить метрики на коротких, средних и длинных предложениях. Если средних/длинных предложений нет в выборке -- сгенерировать самостоятельно, например через LLM.

# Загрузка данных

In [1]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

--2025-06-10 20:57:18--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.2.207, 74.125.137.207, 142.250.101.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.2.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.01s   

2025-06-10 20:57:18 (231 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [2]:
text_file = "spa-eng/spa.txt"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
text_pairs = []
for line in lines:
    english, spanish = line.split("\t")
    spanish = "[start] " + spanish + " [end]"
    text_pairs.append((english, spanish))

In [3]:
import random
print(random.choice(text_pairs))

('Get on the bus.', '[start] ¡Súbase al bus! [end]')


In [4]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]

**Векторизация пар**

In [5]:
import tensorflow as tf
import string
import re
from tensorflow import keras
from tensorflow.keras import layers

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace("[", "")
strip_chars = strip_chars.replace("]", "")

def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(
        lowercase, f"[{re.escape(strip_chars)}]", "")

vocab_size = 15000
sequence_length = 20

source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize=custom_standardization,
)
train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

**Подготовка датасетов**

In [6]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [7]:
train_ds

<CacheDataset element_spec=({'english': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'spanish': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

In [8]:
val_ds

<CacheDataset element_spec=({'english': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'spanish': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

In [9]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


In [10]:
train_ds.take(1)

<_TakeDataset element_spec=({'english': TensorSpec(shape=(None, None), dtype=tf.int64, name=None), 'spanish': TensorSpec(shape=(None, None), dtype=tf.int64, name=None)}, TensorSpec(shape=(None, None), dtype=tf.int64, name=None))>

In [11]:
inputs['english']

<tf.Tensor: shape=(64, 20), dtype=int64, numpy=
array([[2783,   35, 2948, ...,    0,    0,    0],
       [  81,  101,  154, ...,    0,    0,    0],
       [  13,  386,  368, ...,    0,    0,    0],
       ...,
       [   6,  569,  248, ...,    0,    0,    0],
       [   5,   63,  678, ...,    0,    0,    0],
       [  40,    3,  101, ...,    0,    0,    0]])>

# RNN сеть

In [12]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)

target_next_step = layers.Dense(vocab_size, activation="softmax")(x)

seq2seq_rnn = keras.Model([source, past_target], target_next_step)
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])

seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 216s 161ms/step - accuracy: 0.1381 - loss: 5.2747 - val_accuracy: 0.1569 - val_loss: 3.9020
Epoch 2/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 214s 164ms/step - accuracy: 0.1592 - loss: 3.9117 - val_accuracy: 0.1882 - val_loss: 3.2610
Epoch 3/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 220s 169ms/step - accuracy: 0.1848 - loss: 3.3435 - val_accuracy: 0.2079 - val_loss: 2.8879
Epoch 4/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 213s 164ms/step - accuracy: 0.2021 - loss: 2.9647 - val_accuracy: 0.2236 - val_loss: 2.6222
Epoch 5/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 219s 168ms/step - accuracy: 0.2165 - loss: 2.6753 - val_accuracy: 0.2347 - val_loss: 2.4352
Epoch 6/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 213s 164ms/step - accuracy: 0.2280 - loss: 2.4405 - val_accuracy: 0.2418 - val_loss: 2.3075
Epoch 7/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 213s 164ms/step - accuracy: 0.2385 - loss: 2.2456 - val_accuracy: 0.2483 - val_loss: 2.2069
Epoch 8/15
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 219s 168ms/step - ac

## RNN + Attention

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
import unicodedata
import re
import random
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

# Предобработка

In [14]:
def normalize(s):
    s = unicodedata.normalize('NFD', s)
    s = s.encode('ascii', 'ignore').decode('utf-8')
    s = re.sub(r'[^\w\s]', '', s)
    s = s.lower().strip()
    return s

def load_data(path):
    pairs = []
    with open(path, 'r', encoding='utf-8') as f:
        for line in f:
            eng, spa = line.strip().split('\t')
            eng = normalize(eng)
            spa = normalize(spa)
            pairs.append((eng, spa))
    return pairs

pairs = load_data('spa-eng/spa.txt')

# Токенизация
def tokenize(pairs):
    input_vocab = set()
    output_vocab = set()
    for en, es in pairs:
        input_vocab.update(en.split())
        output_vocab.update(es.split())
    input_vocab = ['<pad>', '<sos>', '<eos>'] + sorted(input_vocab)
    output_vocab = ['<pad>', '<sos>', '<eos>'] + sorted(output_vocab)
    input_word2idx = {word: i for i, word in enumerate(input_vocab)}
    output_word2idx = {word: i for i, word in enumerate(output_vocab)}
    input_idx2word = {i: word for word, i in input_word2idx.items()}
    output_idx2word = {i: word for word, i in output_word2idx.items()}
    return input_word2idx, output_word2idx, input_idx2word, output_idx2word, input_vocab, output_vocab

input_w2i, output_w2i, input_i2w, output_i2w, input_vocab, output_vocab = tokenize(pairs)

MAX_LEN = 10

def encode_sentence(sentence, word2idx, max_len):
    tokens = sentence.split()
    tokens = tokens[:max_len-2]
    tokens = ['<sos>'] + tokens + ['<eos>']
    ids = [word2idx.get(tok, 0) for tok in tokens]
    ids += [0] * (max_len - len(ids))
    return ids

# Dataset
class TranslationDataset(Dataset):
    def __init__(self, pairs):
        self.pairs = pairs
        self.x = [encode_sentence(p[0], input_w2i, MAX_LEN) for p in pairs]
        self.y = [encode_sentence(p[1], output_w2i, MAX_LEN) for p in pairs]

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        return torch.tensor(self.x[idx]), torch.tensor(self.y[idx])

train_pairs, val_pairs = train_test_split(pairs, test_size=0.1)
train_data = TranslationDataset(train_pairs)
val_data = TranslationDataset(val_pairs)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32)

# Модель

In [15]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super().__init__()
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.gru = nn.GRU(emb_dim, hid_dim, batch_first=True)

    def forward(self, src):
        embedded = self.embedding(src)
        outputs, hidden = self.gru(embedded)
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        self.attn = nn.Linear(hid_dim * 2, hid_dim)
        self.v = nn.Parameter(torch.rand(hid_dim))

    def forward(self, hidden, encoder_outputs):
        src_len = encoder_outputs.size(1)
        hidden = hidden[-1].unsqueeze(1).repeat(1, src_len, 1)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))
        energy = energy @ self.v
        return torch.softmax(energy, dim=1)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, attention):
        super().__init__()
        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.gru = nn.GRU(hid_dim + emb_dim, hid_dim, batch_first=True)
        self.fc_out = nn.Linear(hid_dim * 2, output_dim)

    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(1)
        embedded = self.embedding(input)
        attn_weights = self.attention(hidden, encoder_outputs)
        context = attn_weights.unsqueeze(1).bmm(encoder_outputs)
        rnn_input = torch.cat((embedded, context), dim=2)
        output, hidden = self.gru(rnn_input, hidden)
        prediction = self.fc_out(torch.cat((output.squeeze(1), context.squeeze(1)), dim=1))
        return prediction, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = src.shape[0]
        trg_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim
        outputs = torch.zeros(batch_size, trg_len, trg_vocab_size).to(self.device)

        encoder_outputs, hidden = self.encoder(src)
        input = trg[:, 0]

        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[:, t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.argmax(1)
            input = trg[:, t] if teacher_force else top1

        return outputs


# Обучение

In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
INPUT_DIM = len(input_vocab)
OUTPUT_DIM = len(output_vocab)
ENC_EMB_DIM = 64
DEC_EMB_DIM = 64
HID_DIM = 128
EPOCHS = 1

attn = Attention(HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, attn)

model = Seq2Seq(enc, dec, device).to(device)
optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index=0)

def train(model, loader):
    model.train()
    epoch_loss = 0
    for src, trg in loader:
        src, trg = src.to(device), trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output_dim = output.shape[-1]
        output = output[:, 1:].reshape(-1, output_dim)
        trg = trg[:, 1:].reshape(-1)
        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(loader)

for epoch in range(1, EPOCHS+1):
    loss = train(model, train_loader)
    print(f'Epoch {epoch}, Loss: {loss:.4f}')

Epoch 1, Loss: 4.0969


# Перевод

In [17]:
def translate_sentence(sentence, model, input_w2i, output_i2w, max_len=MAX_LEN):
    model.eval()
    with torch.no_grad():
        sentence = normalize(sentence)
        tokens = sentence.split()
        tokens = ['<sos>'] + tokens + ['<eos>']
        ids = [input_w2i.get(tok, 0) for tok in tokens]
        ids += [0] * (max_len - len(ids))
        src_tensor = torch.tensor(ids).unsqueeze(0).to(device)

        encoder_outputs, hidden = model.encoder(src_tensor)
        input_tok = torch.tensor([output_w2i['<sos>']]).to(device)

        outputs = []
        for _ in range(max_len):
            output, hidden = model.decoder(input_tok, hidden, encoder_outputs)
            top1 = output.argmax(1)
            word = output_i2w[top1.item()]
            if word == '<eos>':
                break
            outputs.append(word)
            input_tok = top1
        return ' '.join(outputs)

Тест

In [18]:
print(translate_sentence("Help me please", model, input_w2i, output_i2w))

ayudame por favor


# BLEU

In [19]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.4 MB/s eta 0:00:00


In [20]:
import sacrebleu

def evaluate_bleu_sacrebleu(model, dataset):
    refs = []
    hyps = []
    for i in range(len(dataset)):
        src_tensor, trg_tensor = dataset[i]
        src_sentence = ' '.join([input_i2w[idx.item()] for idx in src_tensor if idx.item() not in [0]])
        trg_sentence = ' '.join([output_i2w[idx.item()] for idx in trg_tensor if idx.item() not in [0, output_w2i['<sos>'], output_w2i['<eos>']]])

        pred = translate_sentence(src_sentence, model, input_w2i, output_i2w)
        if pred.strip() and trg_sentence.strip():
            refs.append(trg_sentence)
            hyps.append(pred)

    bleu = sacrebleu.corpus_bleu(hyps, [refs])
    return bleu.score

In [21]:
bleu = evaluate_bleu_sacrebleu(model, val_data)
print(f'BLEU: {bleu:.4f}')

BLEU: 13.1141


**Пример перевода с помощью RNN сети**

In [22]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_predictions = seq2seq_rnn.predict(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
The birds are singing.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
[start] los pájaros están cantando [end]
-
Tangerines contain a lot of vitamin C.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
[start] las [UNK] contienen mucha vitamina c [end]
-
His brother went to the station to see him off.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36m

# RNN + attention

# Трансформер

**Класс `TransformerDecoder`**

In [23]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        else:
            padding_mask = mask
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)

**Слой PositionalEmbedding**

In [24]:
import keras.ops as ops

In [25]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, vocab_size, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=embed_dim
        )
        self.sequence_length = sequence_length
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim

    def call(self, inputs):
        length = ops.shape(inputs)[-1]
        positions = ops.arange(0, length, 1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions

    def compute_mask(self, inputs, mask=None):
        return ops.not_equal(inputs, 0)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "sequence_length": self.sequence_length,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

**End-to-end Трансформер**

In [26]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [27]:
embed_dim = 256
dense_dim = 2048
num_heads = 8


encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)

decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:938: UserWarning: Layer 'transformer_encoder' (of type TransformerEncoder) was passed an input with a mask attached to it. However, this layer does not support masking and will therefore destroy the mask information. Downstream layers will not see the mask.
  warnings.warn(


**Обучение**

In [28]:
transformer.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 98s 64ms/step - accuracy: 0.1605 - loss: 4.3914 - val_accuracy: 0.2494 - val_loss: 2.2407
Epoch 2/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 75s 57ms/step - accuracy: 0.2510 - loss: 2.2887 - val_accuracy: 0.2758 - val_loss: 1.7351
Epoch 3/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 82s 57ms/step - accuracy: 0.2786 - loss: 1.7090 - val_accuracy: 0.2843 - val_loss: 1.5847
Epoch 4/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 73s 56ms/step - accuracy: 0.2927 - loss: 1.4142 - val_accuracy: 0.2895 - val_loss: 1.4822
Epoch 5/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 73s 56ms/step - accuracy: 0.3027 - loss: 1.2192 - val_accuracy: 0.2920 - val_loss: 1.4479
Epoch 6/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 73s 56ms/step - accuracy: 0.3108 - loss: 1.0807 - val_accuracy: 0.2937 - val_loss: 1.4427
Epoch 7/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 73s 56ms/step - accuracy: 0.3166 - loss: 0.9806 - val_accuracy: 0.2953 - val_loss: 1.4542
Epoch 8/30
1302/1302 ━━━━━━━━━━━━━━━━━━━━ 73s 56ms/step - accuracy: 0.3219 -

**Пример перевода**

In [29]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]"
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
My door is always open.
[start] mi puerta está siempre abierta [end]
-
How about dining out tonight?
[start] qué tal tal esta noche [end]
-
We made inquiries into his past.
[start] hicimos pocas posibilidades en su pasado [end]
-
We're not the only ones here who know Tom.
[start] no somos los únicos a tom que saben [end]
-
My uncle drives a Ford.
[start] mi tío conduce un ford [end]
-
What did you want to talk to me about?
[start] de qué querías hablar conmigo [end]
-
What did I leave behind?
[start] qué me dejé atrás [end]
-
She wouldn't speak to him.
[start] ella no le [UNK] [end]
-
I feel their pain.
[start] siento su dolor [end]
-
Tom loves his job.
[start] tom ama su trabajo [end]
-
The sky is becoming cloudy.
[start] el cielo se está poniendo nublado [end]
-
Tom went to see Mary the following Friday.
[start] tom fue a ver a mary al viernes [end]
-
Tom and Mary were snuggled up together watching TV when John knocked on the door.
[start] tom y mary estaban acurrucados juntos de j